In [106]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import re

In [107]:
df = pd.read_csv('test.csv')
dft = pd.read_csv('todasCenario1.csv')

In [108]:
df.columns

Index(['full_name', 'registered_year', 'engine_capacity', 'insurance',
       'transmission_type', 'kms_driven', 'owner_type', 'fuel_type',
       'max_power', 'seats', 'mileage', 'body_type', 'city', 'ID'],
      dtype='object')

In [109]:
dft.columns


Index(['marca', 'id_marcas', 'engine_capacity', 'id_insurance', 'insurance',
       'id_transmissao', 'transmission_type', 'ano', 'kms_driven',
       'owner_type', 'id_owner_type', 'fuel_type', 'id_fuel_type', 'max_power',
       'seats', 'mileage', 'id_body_type', 'body_type', 'id_city', 'city',
       'resale_price_Lakh'],
      dtype='object')

In [110]:
#MILEAGE_____________________________________________________

#assign value to blank lines
df['mileage'] = df['mileage'].fillna(0)

#remove and convert 'kmpl' and 'km/kg/ from  mileage
df = df[df['mileage'].notna()]
consumo_correto = []

for i in df['mileage']:
    match = re.search(r'(\d+\.\d+|\d+)', str(i))  # Encontrar números (com ou sem ponto decimal)
    
    if match:
        valor = float(match.group())
        
        if 'km/kg' in str(i):
            valor *= 1.40  # Converter de km/kg para kmpl
        
        consumo_correto.append(valor)

df['mileage'] = consumo_correto

In [111]:
# KMS __________________________________________________
#remove kms from kms_driven
df['kms_driven'] = df['kms_driven'].str.replace('Kms', '')
# Remove , and blanks from 'kms_driven'
df['kms_driven'] = df['kms_driven'].str.replace(',', '').str.strip()

In [112]:
# ENGINE __________________________________________________
#remove cc from engine_capacity
df['engine_capacity'] = df['engine_capacity'].str.replace('cc', '')

In [113]:
# Substituir os valores 'Third Party insurance' por 'Third Party' em df
df['insurance'] = df['insurance'].replace('Third Party insurance', 'Third Party', regex=True)

# Substituir os valores NaN por 'No Insurance' em df
df['insurance'].fillna('No Insurance', inplace=True)

# Visualizar a coluna 'insurance' atualizada em df
print(df['insurance'].unique())
print(dft['id_insurance'].unique())

['Comprehensive' 'Not Available' 'Third Party' 'Zero Dep' 'No Insurance']
[0.25663332 1.30640077 0.44540821 1.41864433 3.34976304]


In [114]:
import pandas as pd
import re

# Supondo que já carregou o DataFrame df

# Função para extrair o ano dos primeiros quatro caracteres de 'full_name' em df
def extrair_ano(nome):
    match = re.match(r'\d{4}', nome)  # Expressão regular para pegar os primeiros quatro dígitos (ano)
    if match:
        return match.group()
    else:
        return None

# Aplicar a função para extrair o ano e criar uma nova coluna 'ano' em df
df['ano'] = df['full_name'].apply(extrair_ano)

# Visualizar o DataFrame df com a nova coluna 'ano'
print(df)


                                           full_name registered_year  \
0     2017 Hyundai Grand i10 1.2 Kappa Sportz Option        Jun 2017   
1                          2020 Maruti Swift AMT VXI        Feb 2021   
2                    2010 Maruti Zen Estilo LX BSIII        Jul 2010   
3                               2010 Maruti Alto LXi            2010   
4                   2018 Hyundai i20 Active S Petrol        Oct 2018   
...                                              ...             ...   
3485                      2022 Tata Tigor EV XZ Plus            2022   
3486                       2017 Renault KWID 1.0 RXT        Apr 2017   
3487                       2017 Honda City i VTEC VX        Dec 2017   
3488                    2016 Honda Jazz 1.2 V i VTEC        Jun 2016   
3489                           2017 Maruti Swift VXI        Mar 2017   

     engine_capacity      insurance transmission_type kms_driven  \
0              1197   Comprehensive            Manual      53658   

In [115]:
import pandas as pd
import re

# Supondo que já carregou os DataFrames df e dft

# Função para extrair a marca a partir da coluna 'full_name' em df
def extrair_marca(nome):
    match = re.search(r'\d{4}\s(.+?)\s', nome)  # Expressão regular para pegar a marca após o ano
    if match:
        return match.group(1)
    else:
        return None

# Aplicar a função para extrair a marca e criar uma nova coluna 'marca' em df
df['marca'] = df['full_name'].apply(extrair_marca)

# Mapear as marcas de df para a coluna 'marca' em dft
mapeamento_marca = dict(zip(dft['marca'], dft['id_marcas']))

# Criar uma nova coluna 'id_categoria_marca' em df com base na correspondência de marcas em dft
df['id_marcas'] = df['marca'].map(mapeamento_marca)

# Visualizar o DataFrame df atualizado
print(df)


                                           full_name registered_year  \
0     2017 Hyundai Grand i10 1.2 Kappa Sportz Option        Jun 2017   
1                          2020 Maruti Swift AMT VXI        Feb 2021   
2                    2010 Maruti Zen Estilo LX BSIII        Jul 2010   
3                               2010 Maruti Alto LXi            2010   
4                   2018 Hyundai i20 Active S Petrol        Oct 2018   
...                                              ...             ...   
3485                      2022 Tata Tigor EV XZ Plus            2022   
3486                       2017 Renault KWID 1.0 RXT        Apr 2017   
3487                       2017 Honda City i VTEC VX        Dec 2017   
3488                    2016 Honda Jazz 1.2 V i VTEC        Jun 2016   
3489                           2017 Maruti Swift VXI        Mar 2017   

     engine_capacity      insurance transmission_type kms_driven  \
0              1197   Comprehensive            Manual      53658   

In [116]:
#soluçoes id_maxpower

import pandas as pd

def adicionar_0_1_maior_valormarca(df, dft):
    # Encontrar o maior valor na coluna 'id_categoria_marca' em dft
    maior_valor_id_max_power_dft = dft['id_marcas'].max()

    # Adicionar 0.1 ao maior valor encontrado
    novo_valor = maior_valor_id_max_power_dft + 0.1

    # Substituir os valores NaN na coluna 'id_categoria_marca' em df pelo novo valor
    df['id_marcas'].fillna(novo_valor, inplace=True)

# Utilização da função com os DataFrames 'df' e 'dft'
adicionar_0_1_maior_valormarca(df, dft)

# Visualizar o DataFrame 'df' atualizado
print(df)

                                           full_name registered_year  \
0     2017 Hyundai Grand i10 1.2 Kappa Sportz Option        Jun 2017   
1                          2020 Maruti Swift AMT VXI        Feb 2021   
2                    2010 Maruti Zen Estilo LX BSIII        Jul 2010   
3                               2010 Maruti Alto LXi            2010   
4                   2018 Hyundai i20 Active S Petrol        Oct 2018   
...                                              ...             ...   
3485                      2022 Tata Tigor EV XZ Plus            2022   
3486                       2017 Renault KWID 1.0 RXT        Apr 2017   
3487                       2017 Honda City i VTEC VX        Dec 2017   
3488                    2016 Honda Jazz 1.2 V i VTEC        Jun 2016   
3489                           2017 Maruti Swift VXI        Mar 2017   

     engine_capacity      insurance transmission_type kms_driven  \
0              1197   Comprehensive            Manual      53658   

In [117]:
# MAX POWER __________________________________________________
#convert to bhp
Correct_Max_Power = []
for power in df['max_power']:
    if pd.notna(power):
        if isinstance(power, str):
            if 'bhp' in power:
                num_bhp = float(re.search(r'\d+.?\d', power).group())
                Correct_Max_Power.append(num_bhp)
            elif 'PS' in power:
                num_ps = float(re.search(r'\d+.?\d', power).group())
                num_bhp = round(num_ps * 0.98632, 2)
                Correct_Max_Power.append(num_bhp)
            else:
                num = re.search(r'\d+\.?\d*', power)
                Correct_Max_Power.append(float(num.group()) if num else None)
        else:
            Correct_Max_Power.append(power)
    else:
        Correct_Max_Power.append(None)


# Convert not NaN to float
Correct_Max_Power = [float(value) if value is not None else value for value in Correct_Max_Power]

df['max_power'] = Correct_Max_Power

In [118]:
import pandas as pd

def adicionar_0_1_maior_valormediaengine(df, dft):
    # Calculando a média dos valores da coluna 'max_power'
    media_engine_capacity = dft['engine_capacity'].mean()

    # Substituir os valores NaN na coluna 'max_power' em df pela média
    df['engine_capacity'].fillna(media_engine_capacity, inplace=True)

    # Resto do seu código...

# Utilização da função com os DataFrames 'df' e 'dft'
adicionar_0_1_maior_valormediaengine(df, dft)

# Visualizar o DataFrame 'df' atualizado
print(df)

                                           full_name registered_year  \
0     2017 Hyundai Grand i10 1.2 Kappa Sportz Option        Jun 2017   
1                          2020 Maruti Swift AMT VXI        Feb 2021   
2                    2010 Maruti Zen Estilo LX BSIII        Jul 2010   
3                               2010 Maruti Alto LXi            2010   
4                   2018 Hyundai i20 Active S Petrol        Oct 2018   
...                                              ...             ...   
3485                      2022 Tata Tigor EV XZ Plus            2022   
3486                       2017 Renault KWID 1.0 RXT        Apr 2017   
3487                       2017 Honda City i VTEC VX        Dec 2017   
3488                    2016 Honda Jazz 1.2 V i VTEC        Jun 2016   
3489                           2017 Maruti Swift VXI        Mar 2017   

     engine_capacity      insurance transmission_type kms_driven  \
0              1197   Comprehensive            Manual      53658   

In [119]:
import pandas as pd

def adicionar_0_1_maior_valormedia(df, dft):
    # Calculando a média dos valores da coluna 'max_power'
    media_max_power = dft['max_power'].mean()

    # Substituir os valores NaN na coluna 'max_power' em df pela média
    df['max_power'].fillna(media_max_power, inplace=True)

    # Resto do seu código...

# Utilização da função com os DataFrames 'df' e 'dft'
adicionar_0_1_maior_valormedia(df, dft)

# Visualizar o DataFrame 'df' atualizado
print(df)


                                           full_name registered_year  \
0     2017 Hyundai Grand i10 1.2 Kappa Sportz Option        Jun 2017   
1                          2020 Maruti Swift AMT VXI        Feb 2021   
2                    2010 Maruti Zen Estilo LX BSIII        Jul 2010   
3                               2010 Maruti Alto LXi            2010   
4                   2018 Hyundai i20 Active S Petrol        Oct 2018   
...                                              ...             ...   
3485                      2022 Tata Tigor EV XZ Plus            2022   
3486                       2017 Renault KWID 1.0 RXT        Apr 2017   
3487                       2017 Honda City i VTEC VX        Dec 2017   
3488                    2016 Honda Jazz 1.2 V i VTEC        Jun 2016   
3489                           2017 Maruti Swift VXI        Mar 2017   

     engine_capacity      insurance transmission_type kms_driven  \
0              1197   Comprehensive            Manual      53658   

In [120]:
# SEATS __________________________________________________
# Check for model cars with missing values in the 'seats' column
distinct_full_names_with_blank_seats = df[df['seats'].isnull()]['full_name'].unique()
print('Distinct full names with blank seats:')
for full_name in distinct_full_names_with_blank_seats:
    print(full_name)

# Update number of seats based on conditions
df.loc[df['full_name'].isin(['2009 Maruti Estilo LXI', '2019 Nissan Kicks Diesel', '2010 Maruti Estilo LXI']), 'seats'] = 5
df.loc[df['full_name'] == '2008 Ford Endeavour 3.0 4x4 Thunder Plus', 'seats'] = 7

# Check
distinct_full_names_with_blank_seats = df[df['seats'].isnull()]['full_name'].unique()
print()
print('Distinct full names with blank seats:')
for full_name in distinct_full_names_with_blank_seats:
    print(full_name)

Distinct full names with blank seats:
2005 Honda City 1.5 EXI
2009 Maruti Estilo LXI
2012 Honda Jazz Select Edition

Distinct full names with blank seats:
2005 Honda City 1.5 EXI
2012 Honda Jazz Select Edition


In [121]:
# Substituir os valores nulos na coluna 'seats' por 5
df['seats'].fillna(5, inplace=True)

In [122]:
# Obtendo os valores únicos na coluna 'owner_type'
valores_unicos_owner_type = df['owner_type'].unique()

# Exibindo os valores únicos
print(valores_unicos_owner_type)

['First Owner' 'Second Owner' 'Third Owner' nan 'Fourth Owner'
 'Fifth Owner']


In [123]:
import pandas as pd

# Supondo que 'df' seja o seu DataFrame

# Substituir os valores NaN por 'First Owner' na coluna 'owner_type'
df['owner_type'].fillna('First Owner', inplace=True)

# Visualizar o DataFrame atualizado
print(df['owner_type'])


0        First Owner
1       Second Owner
2        Third Owner
3        First Owner
4       Second Owner
            ...     
3485     First Owner
3486     First Owner
3487     First Owner
3488     First Owner
3489    Second Owner
Name: owner_type, Length: 3490, dtype: object


In [124]:
# Contar os valores nulos (NaN) em cada coluna
contagem_nan = df.isna().sum()

# Exibir a contagem de valores nulos em cada coluna
print(contagem_nan)

full_name             0
registered_year      15
engine_capacity       0
insurance             0
transmission_type     0
kms_driven            0
owner_type            0
fuel_type             0
max_power             0
seats                 0
mileage               0
body_type             0
city                  0
ID                    0
ano                   0
marca                 0
id_marcas             0
dtype: int64


In [125]:
df

,full_name,registered_year,engine_capacity,insurance,transmission_type,kms_driven,owner_type,fuel_type,max_power,seats,mileage,body_type,city,ID,ano,marca,id_marcas
0,2017 Hyundai Grand i10 1.2 Kappa Sportz Option,Jun 2017,1197,Comprehensive,Manual,53658,First Owner,Petrol,81.8,5.0,18.90,Hatchback,Mumbai,0,2017,Hyundai,0.677665
1,2020 Maruti Swift AMT VXI,Feb 2021,1197,Not Available,Automatic,33055,Second Owner,Petrol,81.8,5.0,21.21,Hatchback,Jaipur,1,2020,Maruti,0.541440
2,2010 Maruti Zen Estilo LX BSIII,Jul 2010,998,Third Party,Manual,90385,Third Owner,Petrol,67.1,5.0,18.20,Hatchback,Chandigarh,2,2010,Maruti,0.541440
3,2010 Maruti Alto LXi,2010,796,Third Party,Manual,110000,First Owner,Petrol,46.3,5.0,19.70,Hatchback,Delhi,3,2010,Maruti,0.541440
4,2018 Hyundai i20 Active S Petrol,Oct 2018,1197,Comprehensive,Manual,58000,Second Owner,Petrol,81.8,5.0,17.19,Hatchback,Delhi,4,2018,Hyundai,0.677665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3485,2022 Tata Tigor EV XZ Plus,2022,1399.651986,Third Party,Automatic,30000,First Owner,Electric,73.7,5.0,0.00,Sedan,Pune,3485,2022,Tata,1.230837
3486,2017 Renault KWID 1.0 RXT,Apr 2017,999,Third Party,Manual,64024,First Owner,Petrol,67.0,5.0,23.01,Hatchback,Bangalore,3486,2017,Renault,1.419493
3487,2017 Honda City i VTEC VX,Dec 2017,1497,Comprehensive,Manual,39000,First Owner,Petrol,117.3,5.0,17.80,Sedan,Delhi,3487,2017,Honda,0.976238
3488,2016 Honda Jazz 1.2 V i VTEC,Jun 2016,1199,Comprehensive,Manual,33000,First Owner,Petrol,88.7,5.0,18.70,Hatchback,Mumbai,3488,2016,Honda,0.976238


In [126]:
#BODY TYPE ----------------------
#Replace the missing types and the ones that were wrong
df.loc[df['full_name'].isin(['2009 Maruti Estilo LXI', '2019 Nissan Kicks Diesel', '2010 Maruti Estilo LXI', '2019 Maruti Wagon R ZXI AMT 1.2BSIV',
                             '2016 Mercedes-Benz B Class B180 Sport', '2016 Mercedes-Benz A Class A200 D Sport Edition',
                             '2004 Tata Indigo Marina GLX', '2017 Volvo V40 D3 R Design']), 'body_type'] = 'Hatchback'
df.loc[df['full_name'].isin(['2005 Tata Sumo Victa EX 10/7 Str BSII', '2008 Tata Sumo CX', '2010 Tata New Safari DICOR 2.2 EX 4x2', 
                             '2017 Mahindra KUV 100 mFALCON G80 K8', '2019 Mahindra Bolero Camper GOLD ZX 2WD']), 'body_type'] = 'SUV'
df.loc[df['full_name'].isin(['2015 Honda Mobilio E i-DTEC', '2018 Maruti Eeco 5 Seater AC BSIV', '2016 Datsun GO Plus A']), 'body_type'] = 'MUV'
df.loc[df['full_name'].isin(['2021 BMW 7 Series 730Ld DPE Signature','2006 Hyundai Accent CRDi', '2007 Maruti SX4 Vxi BSIII',
                             '2020 Mercedes-Benz S-Class Maybach S560', '2004 Skoda Octavia 1.8 Turbo Petrol RS MT']), 'body_type'] = 'Sedan'
df.loc[df['full_name'].isin(['2012 Mercedes-Benz SLK-Class SLK 350', '2013 Mercedes-Benz E-Class E350 Cabriolet', 
                             '2009 Mercedes-Benz SLK-Class 55 AMG']), 'body_type'] = 'Convertibles'
df.loc[df['full_name'] == '2011 Jaguar XK R V8 Coupe', 'body_type'] = 'Coupe'
df.loc[df['full_name'].isin(['2015 Chevrolet Enjoy 1.4 LS 7', '2004 Maruti Versa DX2 8-SEATER BSIII TWIN A/C', '2021 Maruti Eeco 5 STR With AC Plus HTR CNG',
                             '2022 Maruti Eeco 5 Seater AC 2020-2022', '2022 Maruti Eeco 5 Seater AC BSVI', '2017 Maruti Eeco Flexi Green', 
                             '2013 Maruti Omni Limited Edition', '2021 Maruti Eeco CNG 5 Seater AC', '2019 Maruti Eeco CNG 5 Seater AC BSIV', 
                             '2019 Maruti Eeco 5 Seater AC', '2013 Maruti Eeco 5 Seater Standard BSIV', '2015 Maruti Omni CNG', 
                             '2015 Maruti Omni MPI CARGO BSIV', '2016 Maruti Eeco CNG 5 Seater AC BSIV', '2013 Toyota Innova 2.5 E Diesel MS 7-seater',
                             '2015 Chevrolet Enjoy 1.4 LS 7']), 'body_type'] = 'Minivans'
df.loc[df['full_name'].isin(['2014 Tata Xenon XT EX 4X2', '2019 Isuzu D-Max High']), 'body_type'] = 'Pickup'

# Check
distinct_full_names_with_blank_body_type = df[df['body_type'].isnull()]['full_name'].unique()
print('Distinct full names with blank body_type:')
for full_name in distinct_full_names_with_blank_body_type:
    print(full_name)

Distinct full names with blank body_type:


In [127]:
# Obtendo os valores únicos na coluna 'owner_type'
valores_unicos_body_type = df['body_type'].unique()

# Exibindo os valores únicos
print(valores_unicos_body_type)

['Hatchback' 'SUV' 'Tata' 'Sedan' 'MUV' 'Minivans' 'Mahindra' 'Coupe'
 'Pickup' 'Cars' 'Maruti' 'Audi' 'Toyota' 'Honda' 'BMW' 'Datsun'
 'Chevrolet' 'Jaguar' 'Porsche']


In [128]:
#soluçoes id_body_type

import pandas as pd

def adicionar_0_1_maior_valormax(df, dft):
    # Encontrar o maior valor na coluna 'id_body_type' em dft
    maior_valor_id_max_power_dft = dft['id_body_type'].max()

    # Adicionar 0.1 ao maior valor encontrado
    novo_valor = maior_valor_id_max_power_dft + 0.1

    # Valores a substituir e seus respectivos substitutos
    substitute_values = {
        'Tata': novo_valor,
        'Mahindra': novo_valor,
        'Maruti': novo_valor,
        'Audi': novo_valor,
        'Toyota': novo_valor,
        'Honda': novo_valor,
        'BMW': novo_valor,
        'Datsun': novo_valor,
        'Chevrolet': novo_valor,
        'Jaguar': novo_valor,
        'Porsche': novo_valor
    }

    # Substituir os valores na coluna 'body_type' pelos valores do dicionário substitute_values
    df['body_type'].replace(substitute_values, inplace=True)



# Utilização da função com os DataFrames 'df' e 'dft'
adicionar_0_1_maior_valormax(df, dft)

# Visualizar o DataFrame 'df' atualizado
print(df)

                                           full_name registered_year  \
0     2017 Hyundai Grand i10 1.2 Kappa Sportz Option        Jun 2017   
1                          2020 Maruti Swift AMT VXI        Feb 2021   
2                    2010 Maruti Zen Estilo LX BSIII        Jul 2010   
3                               2010 Maruti Alto LXi            2010   
4                   2018 Hyundai i20 Active S Petrol        Oct 2018   
...                                              ...             ...   
3485                      2022 Tata Tigor EV XZ Plus            2022   
3486                       2017 Renault KWID 1.0 RXT        Apr 2017   
3487                       2017 Honda City i VTEC VX        Dec 2017   
3488                    2016 Honda Jazz 1.2 V i VTEC        Jun 2016   
3489                           2017 Maruti Swift VXI        Mar 2017   

     engine_capacity      insurance transmission_type kms_driven  \
0              1197   Comprehensive            Manual      53658   

In [129]:
import pandas as pd

def substituir_valores(df, coluna_alvo, mapeamento):
    df[coluna_alvo] = df[coluna_alvo].replace(mapeamento).fillna(df[coluna_alvo])



# Criar os mapeamentos desejados para cada coluna
mapeamento_id_cidade = dict(zip(dft['city'], dft['id_city']))
mapeamento_id_body_type = dict(zip(dft['body_type'], dft['id_body_type']))
mapeamento_id_owner_type = dict(zip(dft['owner_type'], dft['id_owner_type']))
mapeamento_id_fuel_type = dict(zip(dft['fuel_type'], dft['id_fuel_type']))
mapeamento_id_insurance = dict(zip(dft['insurance'], dft['id_insurance']))
mapeamento_id_tranmission = dict(zip(dft['transmission_type'], dft['id_transmissao']))




# Chamar a função para substituir os valores nas respectivas colunas
substituir_valores(df, 'city', mapeamento_id_cidade)
substituir_valores(df, 'body_type', mapeamento_id_body_type)
substituir_valores(df, 'owner_type', mapeamento_id_owner_type)
substituir_valores(df, 'fuel_type', mapeamento_id_fuel_type)
substituir_valores(df, 'insurance', mapeamento_id_insurance)
substituir_valores(df, 'transmission_type', mapeamento_id_tranmission)


# Visualizar o DataFrame 'df' atualizado
print(df)

                                           full_name registered_year  \
0     2017 Hyundai Grand i10 1.2 Kappa Sportz Option        Jun 2017   
1                          2020 Maruti Swift AMT VXI        Feb 2021   
2                    2010 Maruti Zen Estilo LX BSIII        Jul 2010   
3                               2010 Maruti Alto LXi            2010   
4                   2018 Hyundai i20 Active S Petrol        Oct 2018   
...                                              ...             ...   
3485                      2022 Tata Tigor EV XZ Plus            2022   
3486                       2017 Renault KWID 1.0 RXT        Apr 2017   
3487                       2017 Honda City i VTEC VX        Dec 2017   
3488                    2016 Honda Jazz 1.2 V i VTEC        Jun 2016   
3489                           2017 Maruti Swift VXI        Mar 2017   

     engine_capacity  insurance  transmission_type kms_driven  owner_type  \
0              1197    0.445408           0.131359      53

In [130]:
# Check data types
data_types = df.dtypes
print("Data type by column:")
print(data_types)

Data type by column:
full_name             object
registered_year       object
engine_capacity       object
insurance            float64
transmission_type    float64
kms_driven            object
owner_type           float64
fuel_type            float64
max_power            float64
seats                float64
mileage              float64
body_type            float64
city                 float64
ID                     int64
ano                   object
marca                 object
id_marcas            float64
dtype: object


In [131]:
# Supondo que 'df' seja o seu DataFrame

# Selecionar colunas para conversão para tipo float
colunas_para_float = ['body_type', 'engine_capacity', 'insurance', 'transmission_type', 'kms_driven',
                      'owner_type', 'fuel_type', 'max_power', 'seats', 'mileage', 'city', 'id_marcas', 'ano']

# Converter colunas para tipo float
df[colunas_para_float] = df[colunas_para_float].astype(float)

# Verificar o tipo das colunas após a conversão
print(df.dtypes)


full_name             object
registered_year       object
engine_capacity      float64
insurance            float64
transmission_type    float64
kms_driven           float64
owner_type           float64
fuel_type            float64
max_power            float64
seats                float64
mileage              float64
body_type            float64
city                 float64
ID                     int64
ano                  float64
marca                 object
id_marcas            float64
dtype: object


In [132]:
# Exportar colunas selecionadas para um novo DataFrame
colunas_selecionadas = ['id_marcas', 'ano', 'engine_capacity', 'insurance', 'transmission_type', 'kms_driven', 'owner_type', 'fuel_type', 'max_power', 'seats', 'mileage', 'body_type', 'city', 'ID']


df_selecionado = df[colunas_selecionadas]
df_selecionado.to_csv('testcenario1.csv', index=False)
 

In [133]:
print(df_selecionado.dtypes)

id_marcas            float64
ano                  float64
engine_capacity      float64
insurance            float64
transmission_type    float64
kms_driven           float64
owner_type           float64
fuel_type            float64
max_power            float64
seats                float64
mileage              float64
body_type            float64
city                 float64
ID                     int64
dtype: object
